## Importing packages

In [ ]:
'''# Record of additions to Jupyter docker's Python packages:
pip install --upgrade pip
pip install lxml
pip install pyxDamerauLevenshtein'''

In [1]:
from bs4 import BeautifulSoup
import pandas, csv, numpy as np
import urllib, requests
import os, os.path, re
from glob import glob

# for comparing strings in order to match elements between lists:
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance as dldist
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance_ndarray as dldistarray

#from lxml import html, etree
# from requests.auth import HTTPBasicAuth, HTTPDigestAuth
# import nbimporter #need to pip install this
# import EncapsulatedSchoolObject as school

## Reading in data

In [2]:
sample = [] # make empty list to store the dictionaries in
with open('../charter_URLs_Apr17.csv', 'r', encoding = 'Latin-1')\
as csvfile: # open file                      
    reader = csv.DictReader(csvfile) # create a reader
    for row in reader: # loop through rows
        sample.append(row) # append each row to the list

In [3]:
# Take a look at the first entry's contents and the variables list in our sample (a list of dictionaries)
print(sample[1]['SEARCH'], "\n", sample[1]["OLD_URL"], "\n")
print(sample[1].keys())

POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL 3425 WINTER LK RD LAC1200, WINTER HAVEN, FL 33881 
 https://www.polk.edu/charter-high-schools/ 

dict_keys(['STABR', 'NCESSCH', 'ADDRESS', 'SCH_NAME', 'SEARCH', 'MANUAL_URL', 'OLD_URL'])


In [ ]:
'''# read in data
df = pandas.read_excel('micro-sample_MANUAL_Apr17_rev3.xlsx')

#get the values from the URL column (stored as list)
links = df['URL'].values

# Input test URL
url = links[0]
url
'''

In [56]:
'''for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep ' 1$'  | wc -l
   55  find www.polk.edu/
   56  find | head
   57  find | head -n 100
   58  find
   59  find www.hustonacademy.com/
   60  find www.hustonacademy.com/ -type d
   61  find www.hustonacademy.com/ -maxdepth 1 -type d
   62  find www.hustonacademy.com/ -maxdepth 0 -type d
   63  find www.hustonacademy.com/ -maxdepth 1 -type d
   64  find www.hustonacademy.com/ -maxdepth 2 -type d
   65  find www.hustonacademy.com/ -maxdepth 3 -type d
   66  find -maxdepth 1 -type d
   67  l s-la
   68  ls -la
   69  for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep ' 1$'  | wc -l
   70  for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep ' 1$'  
   71  for i in $(find -maxdepth 1 -type d); do echo -n "$i "; find $i -type f | wc -l; done  | grep -v ' 1$'

!find .  -maxdepth 1 -type d | wc -l

!find /tmp  -maxdepth 1 -type d | wc -l'''

277


## Parsing soup for ALL TEXT

In [9]:
# Use for-loop with HTML in directories to create list of tuples, where first element is school name
# and second element is a list of strings, each string being the text extracted from each school's site. 
# Step 1: loop through directories
# Step 2: locate HTML files
# Step 3: use BS (or XPATH?) to extract text from HTML
# Step 4: append the text to the dict(??)
# Step 5: Use directory name as first element of tuple, text as second element

In [5]:
os.getcwd()

'/home/jovyan/work/Charter-school-identities/scripts'

In [6]:
testurl = "https://acaciamiddle.org/about/our-mission-and-philosophy"

try:
    testfile = "/home/jovyan/work/mirror/acaciamiddle.org/about/our-mission-and-philosophy.html"
    testfolder = "/home/jovyan/work/mirror/acaciamiddle.org/"
    soup = BeautifulSoup(open(testfile), "lxml")
except:
    html_page = urllib.request.urlopen(testurl)
    soup = BeautifulSoup(html_page, "lxml")

ptags = soup.find_all("p")
print(ptags)

[<p>The mission of Acacia Middle Charter School is to provide a nurturing and challenging learning community to the greater Stockton area. Students will experience learning in ways that will allow them to both understand and improve their world. Our exceptional teachers inspire students to seek deep, meaningful understanding; to develop strong academic skills; and to become creative and independent thinkers. A rapidly changing global environment requires that Acacia students acquire knowledge and develop 21<sup>st</sup> century skills, while learning to define and solve problems creatively. Acacia Middle cultivates compassion, fairness, respect, and individuality within a community that celebrates diversity, authenticity, and service.</p>, <p>Acacia Middle embodies the following key assumptions in philosophy and daily practice:</p>, <p> </p>, <p> </p>, <p> </p>, <p><span style="font-size:14px;"><strong>(209) 956-2182 Fax</strong></span></p>, <p>©Tri Valley Learning Corporation - All Ri

In [82]:
'''# This gives only the VISIBLE text on the web page:

from bs4.element import Comment
data = soup.findAll(text=True) 

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title', 'meta']:
        return False
    elif isinstance(element, Comment):
        return False
    return True

# Alternative way of ripping out comments (line 3 of "visible" function above):
# elif re.match('<!--.*-->', str(element.encode('utf-8'))):

visible_texts = list(filter(visible, data))
visible_texts'''

In [9]:
# For different ways of parsing the text, identify elements we know we DON'T want:
junkName = ['style', 'script', '[document]', 'head', 'title', 'meta', 'a', 'label', 'footer']
# And also elements we know we DO want:
correctName = ['p', 'li', 'table', 'ul', 'ol', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9', 'h10']

In [11]:
def rip_text(fname, junkName_list):

    '''This method of grabbing the visible text from a .html file first converts the HTML to a BeautifulSoup object, 
    then it rips out all script and style elements with tags identified through junkName_list. 
    It finishes by taking out the newlines and outputting a single string of all the visible text from the .html file.'''

    soup = BeautifulSoup(open(fname), "lxml")

    for script in soup(junkName_list):   # loop through every junk tag in soup
        script.extract()    # rip out all script and style elements

    ripped_text = soup.get_text() # get text
    # print(ripped_text.encode('utf-8'))   # for checking work

    # break into lines and remove leading and trailing space on each:
    lines = (line.strip() for line in ripped_text.splitlines())
    # break multi-headlines into a line each:
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    ripped_textc = ' '.join(chunk for chunk in chunks if chunk)

    print("\n", ripped_textc.encode('utf-8'))   # print output

In [12]:
rip_text(testfile, junkName)


 b'Main menu Our Mission and Philosophy English Our Mission: The mission of Acacia Middle Charter School is to provide a nurturing and challenging learning community to the greater Stockton area. Students will experience learning in ways that will allow them to both understand and improve their world.\xc2\xa0Our exceptional teachers inspire students to seek deep, meaningful understanding; to develop strong academic skills; and to become creative and independent thinkers. A rapidly changing global environment requires that Acacia students acquire knowledge and develop 21st century skills, while learning to define and solve problems creatively. Acacia Middle cultivates compassion, fairness, respect, and individuality within a community that celebrates diversity, authenticity, and service. Our Philosophy: Acacia Middle embodies the following key assumptions in philosophy and daily practice: learning is the young person\xe2\x80\x99s pathway to personal growth school should be both challen

In [13]:
def keep_text(fname, correctName_list):

    '''This method of grabbing the visible text from a .html file first converts the HTML to a BeautifulSoup object, 
    then it keeps only those elements with tags identified through correctName_list. 
    It finishes by taking out the newlines and outputting a single string of all the visible text from the .html file.'''

    soup = BeautifulSoup(open(fname), "lxml")

    for script in soup:   # loop through every tag in the soup object
        if script not in soup(correctName_list):
            script.extract()    # keep only the good tags
        
    good_text = soup.get_text() # get text
    # print(ripped_text.encode('utf-8'))   # for checking work

    # break into lines and remove leading and trailing space on each:
    lines = (line.strip() for line in good_text.splitlines())
    # break multi-headlines into a line each:
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    good_textc = ' '.join(chunk for chunk in chunks if chunk)

    print("\n", good_textc.encode('utf-8'))   # print output

In [14]:
keep_text(testfile, correctName)


 b'Our Mission and Philosophy | Acacia Middle Charter School <!--//--><![CDATA[//><!-- jQuery.extend(Drupal.settings, {"basePath":"\\/","pathPrefix":"","ajaxPageState":{"theme":"lvcp","theme_token":"Vg8IZ5kokc7eZAY3hPUqB-Td53PVSK-EF5s11P2_Jdg","js":{"misc\\/jquery.js":1,"misc\\/jquery.once.js":1,"misc\\/drupal.js":1,"sites\\/all\\/libraries\\/colorbox\\/jquery.colorbox-min.js":1,"sites\\/all\\/modules\\/colorbox\\/js\\/colorbox.js":1,"sites\\/all\\/modules\\/colorbox\\/styles\\/default\\/colorbox_style.js":1,"sites\\/all\\/modules\\/field_group\\/field_group.js":1,"sites\\/all\\/themes\\/noodle\\/base\\/noodle.js":1,"sites\\/all\\/libraries\\/superfish\\/superfish.js":1,"sites\\/all\\/libraries\\/superfish\\/jquery.hoverIntent.minified.js":1,"sites\\/all\\/libraries\\/superfish\\/supersubs.js":1,"sites\\/all\\/modules\\/superfish\\/superfish.js":1},"css":{"modules\\/system\\/system.base.css":1,"modules\\/system\\/system.menus.css":1,"modules\\/system\\/system.messages.css":1,"modules\

In [15]:
bool(BeautifulSoup(open(testfile), "html.parser").find())==True

True

In [64]:
folder_list = list(os.walk(data_loc))[0][1]
print(len(folder_list))
folder_list

276


['www.stcroixprep.org',
 'www.kawaikini.com',
 'www.confluenceacademy.com',
 'www.shawanoschools.com',
 'www.performanceacademies.com',
 'www.sunvalleycharterschool.com',
 'highlandfreeschool.org',
 'www.janesville.k12.wi.us',
 'www.aascalifornia.org',
 'life.hartlake.org',
 'evolutionacademy.org',
 'www.bcclc.com',
 'www.hustonacademy.com',
 'www.mnohs.org',
 'www.rivervalleyhigh.org',
 'www.nycmcs.org',
 'www.providencehall.com',
 'excellenceboys.uncommonschools.org',
 'www.spacharter.org',
 'www.northwoodcs.org',
 'ca01902906.schoolwires.net',
 'www.lakecountryacademy.com',
 'paintedrockacademy.com',
 'innovationsacademy.org',
 'www.sage-academy.org',
 'www.newcastle.k12.ca.us',
 'www.pvcics.org',
 'bakercharters.org',
 'www.wattslearningcenter.org',
 'rvms.vail.k12.az.us',
 'www.magellancharter.org',
 'www.leadingedgeacademy.com',
 'enlaceacademy.org',
 'efcps.org',
 'www.palcs.org',
 'www.ivyprepacademy.org',
 'responsiveed.com',
 'www.minisacademy.org',
 'www.kippmetroatlanta.org

In [49]:
# TO DO: FIX
# Point to where HTML has been captured
data_loc = "/home/jovyan/work/mirror/"

# Create list of all the folders we grabbed (not files)
folder_list = []
for elem in os.listdir(data_loc):
    if os.path.isdir(elem):
        folder_list.append(elem)

print(len(folder_list))
folder_list

0


[]

In [19]:
URL_list = []
for school in sample:
    if school["MANUAL_URL"] != '':
        URL_list.append(school["MANUAL_URL"])

print(len(URL_list))
URL_list    

280


['https://www.richland2.org/charterhigh/',
 'https://www.polk.edu/lakeland-gateway-to-college-high-school/',
 'https://www.nhaschools.com/schools/rivercity/Pages/default.aspx',
 'https://www.nhaschools.com/schools/detroitenterprise',
 'https://www.minisacademy.org',
 'https://www.lys.org/services/lighthouse-community-school/',
 'https://westlakecharter.com',
 'https://vangoghcs-lausd-ca.schoolloop.com/',
 'https://summitacademies.org/schools/dayton-high/',
 'https://cms.springbranchisd.com/wais/',
 'https://cityacademy.org/',
 'https://cityacademy.org/',
 'https://byrnes-ecsd-fl.schoolloop.com/',
 'https://acaciamiddle.org/',
 'http://www2.milwaukee.k12.wi.us/whittier/whittier.htm',
 'http://www.zoelearningcenters.com',
 'http://www.ywlcs.org/',
 'http://www.yuba.net/',
 'http://www.ysacs.org/',
 'http://www.wpaces.org/',
 'http://www.woodcharter.com/Granbury/',
 'http://www.willowcreekcharter.com/',
 'http://www.whitepineacademy.com/',
 'http://www.westridgeacademycharter.com/',
 'htt

In [65]:
list(set(folder_list).intersection(URL_list))

[]

In [86]:
for folder in folder_list:
    print(URL_list.index(min(dldistarray(folder, np.array(URL_list)))))
#    URL_list.index(URL_list[index_of_above]

ValueError: 0.2962963 is not in list

In [173]:
def parse_text(dictry, folder_name, URL_var): 
    
    # Note: Need to sort folders (in data_location) and schools (in list_of_dicts) on SAME LIST of URLs!
    # can index folder location to dictionary because dict["URL"]=folder_name!
    
    folder_list = list(os.walk(data_loc))[0][1] # Create list of all the folders in data_location
    
    '''folder_list = [] # Faster, but doesn't work now
    for elem in os.listdir(data_loc):
        if os.path.isdir(elem):
            folder_list.append(elem)'''
    
    for folder in folder_list:
        print("\nParsing this folder: " + folder + "...")
        
        for fname in glob(folder + "**", recursive=True):
            try:
                if bool(BeautifulSoup(open(fname), "html.parser").find())==True:
                    print("Parsing HTML in " + fname + "...")
                    soup = BeautifulSoup(open(fname), "lxml")
                    dict["all_text"] = rip_text(fname, junkName)
            except:
                pass
        
        
        # look in each folder for .html, read in the text, sub ' for b' if needed at start string
        # join together all the text strings, add that as value to school dict, analyze as before.
        # keep out all sites with only ONE HTML file

Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/project-based-learning.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/academics/moby-max.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/academics/caaspp-california-assessment.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/academics/typing-club.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/academics/read-180.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/academics/academic-program-overview.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/academics/math-180.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/student-life/community-service.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/student-life/student-life-overview.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/es/empleador-que-empareja.html...
Parsing HTML in /home/jovyan/work/mirror/acaciamiddle.org/es/apoyo-acac

In [43]:
os.defpath?

In [ ]:
parse_text(sample[0], , "MANUAL_URL")

In [77]:
print(dldist('smtih', 'smith'))  # expected result: 0.2

array = np.array(['test1', 'test12', 'test123'])
print('\n', dldistarray('test', array))  # expected result: [0.2, 0.33333334, 0.42857143]
print('\n', min(dldistarray('test', array)))

0.20000000298023224

 [ 0.2         0.33333334  0.42857143]

 0.2


## Parsing soup for keywords

In [7]:
# Define lists of keywords for each category of interest, used to group the text
mission = ['mission',' vision ', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective']
curriculum = ['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure']
philosophy = ['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value',  'moral', 'Values']
history = ['history', 'our story', 'the story', 'school story', 'background', 'founding', 'founded', 'established', 'establishment', 'our school began', 'we began', 'doors opened', 'school opened']
general =  ['about us', 'our school', 'who we are', 'overview', 'general information', 'our identity', 'profile', 'highlights']

keyValues = [mission, curriculum, philosophy, history, general]
nodeParams = ['mission', 'curriculum', 'philosophy', 'history', 'general']

In [8]:
# This helper function creates a list of tags (with text) for a given keyWord:
def findTags(soup, keyWord):
    tagList = []
        
    for elem in soup(text=re.compile(keyWord)):
        tagList.append(elem.parent)
            
    return tagList

In [9]:
mission_tags = findTags(soup, "mission")
mission_tags

[<p>The mission of Acacia Middle Charter School is to provide a nurturing and challenging learning community to the greater Stockton area. Students will experience learning in ways that will allow them to both understand and improve their world. Our exceptional teachers inspire students to seek deep, meaningful understanding; to develop strong academic skills; and to become creative and independent thinkers. A rapidly changing global environment requires that Acacia students acquire knowledge and develop 21<sup>st</sup> century skills, while learning to define and solve problems creatively. Acacia Middle cultivates compassion, fairness, respect, and individuality within a community that celebrates diversity, authenticity, and service.</p>]

## BETA/ FOR REFERENCE (mostly from this notebook)

In [22]:
## Create HTML soup
# AKCAN'S VERSION:
html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "lxml")

# THAO'S VERSION:
html_page = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
soup = BeautifulSoup(html_page.text, "lxml")

In [9]:
#(self, url, nodeParams, keyValues)


#creating a list of all the keyword lists
#keyValues = [mission, curriculum, philosophy, history, target, resources, orgfactor]
keyValues = [mission, curriculum, philosophy, history, general]

#creating a list of key names for the JSON node that correspond to the keyValues list
#nodeParams = ['mission', 'curriculum', 'philosophy', 'history', 'target', 'resources' , 'organizational_factors']
nodeParams = ['mission', 'curriculum', 'philosophy', 'history', 'general']
testedLinks = {}

#print(links[0])



#constructor = school.WebsiteData('https://www.richland2.org/charterhigh/', nodeParams, keyValues)
#constructor.createJsonNode()
#print(constructor.URL)

def tester(links, keyValues, nodeParams):
    testedLinks[links[5]] = school.WebsiteData(links[5], nodeParams, keyValues).createJsonNode()
    #for i in range(5, 6):
        #testedLinks[links[i]] = school.WebsiteData(links[i], nodeParams, keyValues).createJsonNode()
tester(links, keyValues, nodeParams)

NameError: name 'school' is not defined

## TESTING (from EncapsulatedSchoolObject.ipynb)

In [34]:
#Finds all tags within a BeautifulSoup object that's text contains a certain keyword and returns a list of these tags.
def findTags(soup, keyWord):
    tagList = []
        
    for elem in soup(text=re.compile(keyWord)):
        tagList.append(elem.parent)
            
    return tagList

In [59]:
#Utilizes a BeautifulSoup object (soup) to scrape all text that contains the given keyword. Filters for certain tags and does
#not take text from solely header (h) tags, instead takes text from next filtered tag after the header.  
def keyWordScraper(soup, keyWord):
    searchTxt = ''
    correctName = ['p', 'li', 'table', 'ul', 'ol']
    tagList = findTags(soup, keyWord)
    #print(tagList)
    
    #This loop goes through each tag and rips the text from that tag. If that tag is a header, then the text from the next 
    #useful tag is ripped instead. All text is saved in searchTxt
    for tag in tagList:
        #print(tag, "\n")
        if 'h' in tag.name:
            typeTag = type(soup.find('li'))
            current = tag.next_sibling
            while current != None and  not isinstance(current, typeTag) and current.name not in correctName:
                current = current.next_sibling
            if current != None:
                if 'ul' in current.name or 'ol' in current.name:
                    for li in current.findAll('li'):
                        searchTxt = searchTxt + li.text
                else:
                    searchTxt = searchTxt + current.text
        #elif url extension includes the keyWord, grab all the text?
        else:
            if(tag.name in correctName):
                    searchTxt = searchTxt + tag.text
                
        return searchTxt

In [63]:
#Applies the keyWordScraper function to a BeautifulSoup object(soup) for a list of keywords. Returns the text found.
def mapKeywordScraper(soup, keyWords):
    foundTxt = ''
    
    for keyWord in keyWords:
        foundTxt = foundTxt + str(keyWordScraper(soup, keyWord))
    #print(foundTxt)
    return foundTxt

In [61]:
def createJsonNode():
    JSONNode = {}
    
    for x in range(0, len(keyValues)):
        print(keyValues[x], "\n")
        JSONNode[nodeParams[x]] = [mapKeywordScraper(soup, keyValues[x])]
        print(JSONNode)
    return JSONNode

In [62]:
createJsonNode()

['mission', ' vision ', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective'] 

NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone
{'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone']}
['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure'] 

NoneNoneNoneNoneNoneNoneNoneNoneNoneNone
{'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone'], 'curriculum': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone']}
['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value', 'moral', 'Values'] 

NoneNoneNoneNoneNoneNoneNoneNoneNone
{'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone'], 'curriculum': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone'], 'philosophy': ['NoneNoneNoneNoneNoneNoneNoneNoneNone']}
['history', 'our story', 'the story', 'school story', 'background', 'founding', 'founded', 'established', 'establishment', 'our school began',

{'curriculum': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNone'],
 'general': ['NoneNoneNoneNoneNoneNoneNoneNone'],
 'history': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone'],
 'mission': ['NoneNoneNoneNoneNoneNoneNoneNoneNoneNoneNone'],
 'philosophy': ['NoneNoneNoneNoneNoneNoneNoneNoneNone']}

## CHUNKS OF CODE

In [13]:
# Input test URL here
url = links[0]
url

'https://www.richland2.org/charterhigh/'

In [29]:
html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "lxml")

In [30]:
def findTags(soup, keyWord):
    tagList = []
        
    for elem in soup(text=re.compile(keyWord)):
        tagList.append(elem.parent)
            
    return tagList

In [32]:
tagList = findTags(soup, keyWord)

TypeError: unhashable type: 'list'

In [ ]:
searchTxt = ''
correctName = ['p', 'li', 'table', 'ul', 'ol']
tagList = findTags(soup, keyWord)
    
#This loop goes through each tag and rips the text from that tag. If that tag is a header, then the text from the next 
 #useful tag is ripped instead. All text is saved in searchTxt
for tag in tagList:
    if 'h' in tag.name:
        typeTag = type(soup.find('li'))
        current = tag.next_sibling
        while current != None and  not isinstance(current, typeTag) and current.name not in correctName:
            current = current.next_sibling
        if current != None:
            if 'ul' in current.name or 'ol' in current.name:
                for li in current.findAll('li'):
                    searchTxt = searchTxt + li.text
            else:
                searchTxt = searchTxt + current.text
    #elif url extension includes the keyWord, grab all the text?
    else:
        if(tag.name in correctName):
            searchTxt = searchTxt + tag.text
                
return searchTxt